# Long-Context Dialogue & Key Fact & False Fact Dataset Creation

In [ ]:
# Set n and run all :)
n = 10

### Installations

In [ ]:
!pip install -q openai

### Imports and Initializations

In [ ]:
# OpenAI related imports and initializations
import openai
openai.api_key = # Owen's Hopkins Key

# Google Colab related imports and initializations
from google.colab import drive
drive.mount('/content/drive')

# Paths related to Google Drive
drive_path = "drive/MyDrive/Masters JHU/Thesis/NLP Road To Publishing/"
#drive_path = "/content/drive/MyDrive/NLP Road To Publishing/"
data_path = drive_path + 'Data/'

# Other necessary imports
import pandas as pd
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Original MSC Data

In [ ]:
def load_msc(data_path, start_index=0, end_index=None):
    """
    Loads, processes, and returns a subset of the MSC data.

    Parameters:
    - data_path (str): The path to the dataset.
    - start_index (int): The starting index for the data subset. Default is 0.
    - end_index (int): The ending index for the data subset. If None, loads data till the end. Default is None.

    Returns:
    - DataFrame: A processed subset of the dataset.
    """

    # Load data from the specified path
    df = pd.read_csv(data_path + 'ParlAI MSC/msc_train.csv')

    # If end_index is not provided, load data till the end
    if end_index is None:
        end_index = len(df)

    # Subset the data
    df = df.iloc[start_index:end_index]

    # Rename columns
    df.rename(columns={
        "session_1_persona1":"s1_p1", "session_1_persona2":"s1_p2", "session_1_conversation":"s1_conv",
        "session_2_persona1":"s2_p1", "session_2_persona2":"s2_p2", "session_2_conversation":"s2_conv",
        "session_3_persona1":"s3_p1", "session_3_persona2":"s3_p2", "session_3_conversation":"s3_conv",
        "session_4_persona1":"s4_p1", "session_4_persona2":"s4_p2", "session_4_conversation":"s4_conv",
        "init_persona1":"init_p1", "init_persona2":"init_p2"
    }, inplace=True)

    # Reorder columns
    columns_order = ['s1_conv', 's1_p1', 's1_p2', 's2_conv', 's2_p1', 's2_p2', 's3_conv', 's3_p1', 's3_p2',
                     's4_conv', 's4_p1', 's4_p2', 'init_p1', 'init_p2']
    df = df[columns_order]

    # Concatenate conversations across all sessions with session indicators
    df['full_conv'] = ('Session 1\n' + df['s1_conv'] + '\nSession 2\n' + df['s2_conv'] +
                       '\nSession 3\n' + df['s3_conv'] + '\nSession 4\n' + df['s4_conv'])

    # Assign a unique identifier to each datapoint
    df['id'] = range(start_index + 1, end_index + 1)

    return df

In [ ]:
msc_data = load_msc(data_path, 0, n)
display(msc_data)

,s1_conv,s1_p1,s1_p2,s2_conv,s2_p1,s2_p2,s3_conv,s3_p1,s3_p2,s4_conv,s4_p1,s4_p2,init_p1,init_p2,full_conv,id
0,"Speaker 1: ""Hi! I'm brian. How are you?"" Speak...",My name is Brian. I like to draw instead of ta...,I work in marketing. I worked on marketing for...,"Speaker 1: ""I was listening to Bob Dylan this ...",I enjoy Bob Dylan. the second harry potter boo...,I like Bob Dylan. The first harry potter book ...,"Speaker 1: ""I applied for an internship in art...",I applied for an art department internship. I ...,"I work for a marketing firm,. My marketing fir...","Speaker 1: ""I just got an email from my doctor...",My name is Brian. I like to draw instead of ta...,I work in marketing firm. I worked on marketin...,I wish I could live in a fantasy world. I've l...,I like folk music. I love to hike. My hiking b...,"Session 1\nSpeaker 1: ""Hi! I'm brian. How are ...",1
1,"Speaker 1: ""I love fall! Almost time to break ...",I have leather boots. I have a leather skirt. ...,I like fall. I am in PhD program. I like to r...,"Speaker 1: ""What are you working on in your Ph...",I like to read. I like thriller movies. I enjo...,I am working on preliminary Ed Psych research ...,"Speaker 1: ""How is your study going?"" Speaker ...",I have a job. I go to school. I feel very busy...,I am working on a doing a study for school. I ...,"Speaker 1: ""The coffee shop had caramel cappac...","I have leather boots, leather skirt. I like fa...",I like fall. I like true crime books. I like c...,I am nearly of tall. I drive a ford truck. My ...,I love to read true crime books. I am in grad ...,"Session 1\nSpeaker 1: ""I love fall! Almost tim...",2
2,"Speaker 1: ""I like to drink a lot"" Speaker 2: ...",I like to drink. I like rebel ipas. My favorit...,I have a husband who just left me. I had relat...,"Speaker 1: ""I dressed my chihuahua in a cute o...",I have a chihuahua.,I have dogs. I have kids. I am going through a...,"Speaker 1: ""How are you and the kids doing lat...",I have a dog. I have a guest house and a pool....,I have kids. My kids don't see their father e...,"Speaker 1: ""I know you said your girls loved F...",I like to drink. I like rebel ipas. My favorit...,I am female. I am going through a divorce. I h...,"I eat exclusively meat. In high school, I came...",We have 4 children and 2 dogs. My husband just...,"Session 1\nSpeaker 1: ""I like to drink a lot"" ...",3
3,"Speaker 1: ""Hi! Whats the weather like there? ...",I am italian. I am a well educated but poor it...,I like to paint. I dont like to clean. I love ...,"Speaker 1: ""I just finish cooking my favorite ...",My family likes Italian food. I love pizza. I ...,I love pizza. I am bored by my office job. I w...,"Speaker 1: ""We are getting a new pizza oven at...",I have a job making Pizza. My restaurant just ...,Meat pizza with extra sausage and pepperoni is...,"Speaker 1: ""What kind of crust do you like for...",I am a well educated but poor italian. I love ...,I like to paint. I love watercolors and paint...,I can tell right away if I'm going to like som...,I hate cleaning. I am italian american. I work...,"Session 1\nSpeaker 1: ""Hi! Whats the weather l...",4
4,"Speaker 1: ""How are you doing this lovely even...",I am good at tennis. I do not love tennis. My ...,I love tennis. My favorite color is red. I lik...,"Speaker 1: ""I finally brought my laptop in to ...",My laptop had something wrong with it. I did n...,I have rats. I live in Texas. My laptop is not...,"Speaker 1: ""Did you figure out what is wrong w...",I know some things about fixing computers. I s...,I have a broken laptop. I play tennis. I am a ...,"Speaker 1: ""How is your laptop working? Was I ...",I am good at tennis. I do not love tennis. My ...,I love tennis. I play tennis. My favorite colo...,I have a job as an it agent. My dad works for ...,I love playing tennis. I'm the youngest of 3. ...,"Session 1\nSpeaker 1: ""How are you doing this ...",5
5,"Speaker 1: ""Hi, how are you doing today"" Speak...",I like pizza

### Naturalizing

In [ ]:
def naturalize_conversation(data: pd.DataFrame, data_path: str, file_name: str) -> pd.DataFrame:
    """
    Naturalize the conversation using GPT-4.

    Parameters:
    - data (DataFrame): The data containing conversations to be naturalized.
    - data_path (str): The path to the dataset.
    - file_name (str): The name of the file where the data is or will be stored.

    Returns:
    - DataFrame: A dataframe with naturalized conversations.
    """

    # Load existing data if it exists and extract existing IDs.
    try:
        existing_natural_data = pd.read_excel(data_path + file_name)
        existing_ids = set(existing_natural_data['id'].values)
    except FileNotFoundError:
        existing_ids = set()

    natural = pd.DataFrame(columns=['id', 'full_conv_natural'])

    # Naturalize each conversation in the provided data
    for idx, conv in zip(data['id'], data['full_conv']):
        if idx not in existing_ids:
            naturalize_prompt = (f"Rework an existing dialogue between two characters, making it sound more natural, "
                                f"fluid, and human-like (such that it is indistinguishable from a human conversation). "
                                f"The conversation is split into four distinct sections, each representing a different "
                                f"passage of time (ex. 2 hours later), but the characters and their personalities remain "
                                f"the same. Each session is separated by new lines. Remember, the conversation should be "
                                f"indistinguishable from that of a human.  While reworking the conversation, please ensure "
                                f"the character personas remain consistent throughout and change the character names to "
                                f"Person 1 and Person 2 (if they aren't already). Indicate where each session begins as "
                                f"the input indicates it.  Here is the dialogue to be reworked (between triple quotes):"
                                f"\n\"\"\"\n {conv} \n\"\"\"")
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": naturalize_prompt}]
            )
            new_row = pd.DataFrame({'id': [idx], 'full_conv_natural': [response['choices'][0]['message']['content']]})
            natural = pd.concat([natural, new_row], ignore_index=True)

    return natural

def split_into_sessions(conversation: str) -> list:
    """
    Splits the conversation into individual sessions and ensures formatting consistency.

    Parameters:
    - conversation (str): The entire conversation string.

    Returns:
    - List: A list of formatted individual sessions.
    """

    # Split the conversation into individual sessions
    sessions = conversation.split("Session ")
    sessions = sessions[1:]

    # Format each session
    formatted_sessions = []
    for session in sessions:
        session = re.sub('^\d+\n', '', session)
        persons = re.split("(Person \d+:)", session)
        formatted_session = "".join([p if "Person" in p else p for p in persons]).strip()
        formatted_session = re.sub(r'(?<=["\'])\s*(?=Person)', '\n', formatted_session)
        formatted_sessions.append(formatted_session)

    return formatted_sessions

def append_and_save(data_path: str, file_name: str, natural: pd.DataFrame) -> pd.DataFrame:
    """
    Appends new naturalized conversations to existing data and saves to file.

    Parameters:
    - data_path (str): Path to the dataset.
    - file_name (str): The name of the file where the data is or will be stored.
    - natural (DataFrame): The naturalized data.

    Returns:
    - DataFrame: The appended dataframe.
    """

    # Load existing data if it exists
    try:
        existing_natural_data = pd.read_excel(data_path + file_name)
        combined_data = pd.concat([existing_natural_data, natural], ignore_index=True)
    except FileNotFoundError:
        combined_data = natural

    # Ensure proper order and split conversations into sessions
    combined_data = combined_data.sort_values(by='id').reset_index(drop=True)
    combined_data[['s1_conv', 's2_conv', 's3_conv', 's4_conv']] = combined_data['full_conv_natural'].apply(split_into_sessions).apply(pd.Series)

    # Save the combined data to file
    combined_data.to_excel(data_path + file_name, index=False)

    return combined_data

In [ ]:
natural_file_name = "natural_conv_gpt4.xlsx"
import pandas as pd

while True:
    natural_data = naturalize_conversation(msc_data, data_path + "Augmented GPT4/", natural_file_name)
    if not natural_data.empty:  # If there's new data to be saved
        result_data = append_and_save(data_path + "Augmented GPT4/", natural_file_name, natural_data)
    natural_data = pd.read_excel(data_path + "Augmented GPT4/" + natural_file_name)

    # Check for NaN values in the dataframe
    if natural_data.isna().any().any():
        # Drop rows containing NaN values
        natural_data.dropna(inplace=True)
        # Save dataframe
        natural_data.to_excel(data_path + "Augmented GPT4/" + natural_file_name, index=False)
    else:
        # If there are no NaN values, break the loop
        break

display(natural_data)

,id,full_conv_natural,s1_conv,s2_conv,s3_conv,s4_conv
0,1,"Session 1\nPerson 1: ""Hey there, I'm Brian. Ho...","Person 1: ""Hey there, I'm Brian. How's your da...","Person 1: ""Funny you mention it, I was jamming...","Person 1: ""Guess what? I took your advice and ...","Person 1: ""Speaking of allergies, I just got a..."
1,2,"Session 1\nPerson 1: ""I can't wait for Fall! F...","Person 1: ""I can't wait for Fall! Finally, an ...","Person 1: ""So, what's your doctoral research t...","Person 1: ""How's that PhD study coming along?""...","Person 1: ""Guess what they had at the coffee s..."
2,3,"Session 1\nPerson 1: ""I'm quite partial to a g...","Person 1: ""I'm quite partial to a good drink m...","Person 1: ""The other day I got my chihuahua al...","Person 1: ""So, how are things with you and the...","Person 1: ""Remembered that your girls love Fro..."
3,4,"Session 1\nPerson 1: ""Hello! How's the climate...","Person 1: ""Hello! How's the climate treating y...","Person 1: ""Just wrapped cooking up an Italian ...","Person 1: ""We're installing a new pizza oven a...","Person 1: ""What kind of pizza crust do you pre..."
4,5,"Session 1\nPerson 1: ""Hey, how are you holding...","Person 1: ""Hey, how are you holding up tonight...","Person 1: ""Good news! Finally got my laptop fi...","Person 1: ""Did you manage to diagnose your lap...","Person 1: ""How's the laptop now? Did my interv..."
5,6,"Session 1\nPerson 1: ""Hey, how's your day goin...","Person 1: ""Hey, how's your day going?""\nPerson...","Person 1: ""Watching this concert right now and...","Person 1: ""Ever consider hanging out with othe...","Person 1: ""Checked out your YouTube channel re..."
6,7,"Session 1\nPerson 1: ""Hey, how's your day shap...","Person 1: ""Hey, how's your day shaping up?""\nP...","Person 1: ""Tried out diving this week. Surpris...","Person 1: ""Tell me more about your Australia d...","Person 1: ""Are you set for your upcoming comed..."
7,8,"Session 1\nPerson 1: ""Hey! What's up?"" Person ...","Person 1: ""Hey! What's up?""\nPerson 2: ""Oh, ju...","Person 1: ""Hey, I started listening to Avatar,...","Person 1: ""I actually picked up 'IT' again for...","Person 1: ""Know of any good rock bands droppin..."
8,9,"Session 1\nPerson 1: ""Hey there, thinking of h...","Person 1: ""Hey there, thinking of hitting up a...","Person 1: ""Just letting you know, I made it sa...","Person 1: ""I took your advice and left my frie...","Person 1: ""Caught some Japanese wrestling show..."
9,10,"Session 1\n\nPerson 1: ""I finally tackled that...","Person 1: ""I finally tackled that mess in my h...","Person 1: ""How about we seize a day for a hike...","Person 1: ""Have you explored any parks or vent...","Person 1: ""Absolutely, I love taking a dip whe..."


### Extending

In [ ]:
def extend_from_natural(data: pd.DataFrame, data_path: str, file_name: str) -> pd.DataFrame:
    """
    Extend the conversation using GPT-4 by taking data from the provided DataFrame,
    ensuring not to reprocess data already in extended_conv.xlsx.

    Parameters:
    - data (DataFrame): The data containing conversations to be extended.
    - data_path (str): The path to the directory where the file is or will be stored.
    - file_name (str): The name of the file where the data is or will be stored.

    Returns:
    - DataFrame: A dataframe with extended conversations.
    """

    # Load existing extended data if it exists and extract existing IDs.
    try:
        existing_extended_data = pd.read_excel(data_path + file_name)
        existing_ids = set(existing_extended_data['id'].values)
    except FileNotFoundError:
        existing_ids = set()

    extended = pd.DataFrame(columns=['id', 's1_conv', 's2_conv', 's3_conv', 's4_conv'])

    # Extend each session in the provided data
    for idx, row in data.iterrows():
        if row['id'] not in existing_ids:
            extended_row = {}
            for col in ['s1_conv', 's2_conv', 's3_conv', 's4_conv']:
                extension_prompt = (f"I have a conversation between two people that is split into 4 sessions. "
                                    f"I will give you the conversation for one of the sessions. Lengthen this "
                                    f"session's conversation as much as you possibly can. While doing so, ensure that: "
                                    f"- You continue the conversation from where it left off (do not repeat the given part of the session) "
                                    f"- The conversation remains realistic — it is two human characters having a natural, fluid conversation "
                                    f"- The character personas remain consistent throughout, even though the topics they discuss change "
                                    f"- The dialogue extension is between 400 and 550 words - Never stop the dialogue midway. "
                                    f"It should end at a reasonable point without me having to ask you to continue generating. "
                                    f"Formatting: Start with triple quotes (\"\"\"), then the dialogue extension, and end with triple quotes (\"\"\"). "
                                    f"No other output should be generated (ex. no \"Session extension: \", \"extension: \", etc.) "
                                    f"Here is the conversation for the session (between triple quotes): "
                                    f"\n\"\"\"\n {row[col]} \n\"\"\"")
                print(extension_prompt)
                response = openai.ChatCompletion.create(
                    model="gpt-4",
                    messages=[{"role": "system", "content": extension_prompt}]
                )
                extended_row[col] = response['choices'][0]['message']['content']

            new_row = pd.DataFrame({'id': [row['id']], **extended_row})
            extended = pd.concat([extended, new_row], ignore_index=True)

    # Reformat each session in the extended data
    for col in ['s1_conv', 's2_conv', 's3_conv', 's4_conv']:
        extended[col] = extended[col].apply(reformat_conversation)

    return extended

def reformat_conversation(text: str) -> str:
    """
    Reformats the conversation to ensure consistent formatting.

    Parameters:
    - text (str): The conversation text to be reformatted.

    Returns:
    - str: The reformatted conversation.
    """

    # Extract conversation within the triple quotes
    extracted_data = re.search(r'\"\"\"(.*?)\"\"\"', text, re.DOTALL)
    conversation = extracted_data.group(1).strip() if extracted_data else ""

    # Split and format the conversation by Person
    split_conversation = re.split(r"(Person \d+:)", conversation)
    formatted_conversation = [split_conversation[i] + " " + split_conversation[i+1].strip() for i in range(1, len(split_conversation), 2)]

    return "\n".join(formatted_conversation)

def append_and_save_extended(data_path: str, file_name: str, new_extended_data: pd.DataFrame) -> pd.DataFrame:
    """
    Appends new extended conversations to the existing extended data and saves to file.

    Parameters:
    - data_path (str): Path to the directory where the file is or will be stored.
    - file_name (str): The name of the file where the data is or will be stored.
    - new_extended_data (DataFrame): The new extended data.

    Returns:
    - DataFrame: The appended dataframe.
    """

    # Load existing extended data if it exists
    try:
        existing_extended_data = pd.read_excel(data_path + file_name)
        combined_data = pd.concat([existing_extended_data, new_extended_data], ignore_index=True)
    except FileNotFoundError:
        combined_data = new_extended_data

    # Ensure proper order and save to file
    combined_data = combined_data.sort_values(by='id').reset_index(drop=True)
    combined_data.to_excel(data_path + file_name, index=False)

    return combined_data


In [ ]:
natural_file_name = "natural_conv_gpt4.xlsx"
natural_data = pd.read_excel(data_path + "Augmented GPT4/" + natural_file_name)
extended_file_name = "extended_conv_gpt4.xlsx"

while True:
    extended_data = extend_from_natural(natural_data, data_path + "Augmented GPT4/", extended_file_name)
    if not extended_data.empty:  # If there's new data to be saved
        result_data = append_and_save_extended(data_path + "Augmented GPT4/", extended_file_name, extended_data)
    extended_data = pd.read_excel(data_path + "Augmented GPT4/" + extended_file_name)

    # Check for NaN values in the dataframe
    if extended_data.isna().any().any():
        # Drop rows containing NaN values
        extended_data.dropna(inplace=True)
        # Save dataframe
        extended_data.to_excel(data_path + "Augmented GPT4/" + extended_file_name, index=False)
    else:
        # If there are no NaN values, break the loop
        break


display(extended_data)

,id,s1_conv,s2_conv,s3_conv,s4_conv
0,1,"Person 1: ""I appreciate all genres, though fol...","Person 1: ""An internship, eh? That's actually ...","Person 1: ""It is. But, hey, laughter is the be...","Person 1: ""That's a great idea! Even though th..."
1,2,"Person 1: ""Fair enough. I find a good melodic ...","Person 1: ""Work is... well, work. I'm learning...","Person 1: ""It should be, I also enjoy their an...","Person 1: ""Haha, I think we'd be quite the inn..."
2,3,"Person 1: ""Absolutely! Great companions, just ...","Person 1: ""That's wonderful to hear! You can c...","Person 1: ""Absolutely, wouldn't be movie night...","Person 1: ""Margaritas? Better stock up on the ..."
3,4,"Person 1: ""Thanks. Maybe I'll win big and trea...","Person 1: ""Well, it's never too late for a car...","Person 1: ""You're welcome! That worked out. Ab...","Person 1: ""Sounds like a blast! I'm also inter..."
4,5,"Person 1: ""Pet rats, really? That's unusual. I...","Person 1: ""Yes, it's a traditional Italian dis...","Person 1: ""Ha! Starving, huh? Well, I could wh...","Person 1: ""Hamsters, huh? They're pretty adora..."
5,6,"Person 1: ""Definitely, I'm into working out an...","Person 2: ""Yes! Chicago has its own charm. Rem...","Person 1: ""There’s this food truck that does a...","Person 2: ""Glad to hear it! The raw energy and..."
6,7,"Person 1: ""Ah, I've always loved the comfort a...","Person 1: ""No problem there! We could use some...","Person 2: ""Nothing beats the tranquility of th...","Person 1: ""Actually, yes. In terms of comedy, ..."
7,8,"Person 1: ""Well, I'm actually a sportswriter. ...","Person 1: ""The crowd atmosphere at the games i...","Person 1: ""Oh, absolutely! I've made it a real...","Person 2: ""I bet. Never had one though. What m..."
8,9,"Person 1: ""Yeah, it's pretty fun to watch. Any...","Person 1: ""That's probably a good idea! But it...","Person 1: ""I'd say about once a week or so. It...","Person 1: ""Ha! Them nails probably get their f..."
9,10,"Person 1: ""Ouch, ulcers are no fun, but I get ...","Person 1: ""Funny you should mention it, I've b...","Person 1: ""Actually, yes! I love swimming. It'...","Person 1: ""Definitely, nature offers the most ..."


### Combined Augmented

In [ ]:
def augment_conversations(natural: pd.DataFrame, extended: pd.DataFrame, data_path: str, file_name : str) -> pd.DataFrame:
    """
    Combines conversations from natural and extended dataframes to produce an augmented conversation dataset.

    Parameters:
    - natural (DataFrame): The natural conversations dataset.
    - extended (DataFrame): The extended conversations dataset.
    - data_path (str): The path to the directory where the augmented file is or will be stored.
    - file_name (str): The name of the file where the data is or will be stored.

    Returns:
    - DataFrame: An augmented dataset combining conversations.
    """

    def add_newlines(text):
        text = re.sub(r'(Person [12]:)', r'\n\1', text)
        text = re.sub(r'\n\n', r'\n', text).strip()
        return text

    augmented = pd.DataFrame(columns=['id', 's1_conv', 's2_conv', 's3_conv', 's4_conv', 'full_conv'])
    new_data = natural.copy()

    # Perform the concatenation operation
    for col in ["s1_conv", "s2_conv", "s3_conv", "s4_conv"]:
        new_data[col] = new_data[col] + extended.loc[new_data.index, col]

    # Create the full_conv column
    new_data["full_conv"] = new_data["s1_conv"] + new_data["s2_conv"] + new_data["s3_conv"] + new_data["s4_conv"]

    # Drop the full_conv_natural column as it's no longer needed
    new_data.drop(columns=["full_conv_natural"], inplace=True)

    # Apply the function to add refined newlines to each of the conversation columns
    cols_to_modify = ["s1_conv", "s2_conv", "s3_conv", "s4_conv", "full_conv"]
    for col in cols_to_modify:
      new_data[col] = new_data[col].apply(add_newlines)

    # Concatenate the new data with the existing augmented data
    augmented = pd.concat([augmented, new_data], ignore_index=True)

    # Save the 'augmented' DataFrame back to the file.
    augmented.to_excel(data_path + file_name, index=False)

    return augmented

In [ ]:
natural_file_name = "natural_conv_gpt4.xlsx"
natural_data = pd.read_excel(data_path + "Augmented GPT4/" + natural_file_name)
extended_file_name = "extended_conv_gpt4.xlsx"
extended_data = pd.read_excel(data_path + "Augmented GPT4/" + extended_file_name)
augmented_file_name = "augmented_conv_gpt4.xlsx"
augmented_data = augment_conversations(natural_data, extended_data, data_path + "Augmented GPT4/", augmented_file_name)
augmented_data = pd.read_excel(data_path + "Augmented GPT4/" + augmented_file_name)

display(augmented_data)

,id,s1_conv,s2_conv,s3_conv,s4_conv,full_conv
0,1,"Person 1: ""Hey there, I'm Brian. How's your da...","Person 1: ""Funny you mention it, I was jamming...","Person 1: ""Guess what? I took your advice and ...","Person 1: ""Speaking of allergies, I just got a...","Person 1: ""Hey there, I'm Brian. How's your da..."
1,2,"Person 1: ""I can't wait for Fall! Finally, an ...","Person 1: ""So, what's your doctoral research t...","Person 1: ""How's that PhD study coming along?""...","Person 1: ""Guess what they had at the coffee s...","Person 1: ""I can't wait for Fall! Finally, an ..."
2,3,"Person 1: ""I'm quite partial to a good drink m...","Person 1: ""The other day I got my chihuahua al...","Person 1: ""So, how are things with you and the...","Person 1: ""Remembered that your girls love Fro...","Person 1: ""I'm quite partial to a good drink m..."
3,4,"Person 1: ""Hello! How's the climate treating y...","Person 1: ""Just wrapped cooking up an Italian ...","Person 1: ""We're installing a new pizza oven a...","Person 1: ""What kind of pizza crust do you pre...","Person 1: ""Hello! How's the climate treating y..."
4,5,"Person 1: ""Hey, how are you holding up tonight...","Person 1: ""Good news! Finally got my laptop fi...","Person 1: ""Did you manage to diagnose your lap...","Person 1: ""How's the laptop now? Did my interv...","Person 1: ""Hey, how are you holding up tonight..."
5,6,"Person 1: ""Hey, how's your day going?""\nPerson...","Person 1: ""Watching this concert right now and...","Person 1: ""Ever consider hanging out with othe...","Person 1: ""Checked out your YouTube channel re...","Person 1: ""Hey, how's your day going?""\nPerson..."
6,7,"Person 1: ""Hey, how's your day shaping up?""\nP...","Person 1: ""Tried out diving this week. Surpris...","Person 1: ""Tell me more about your Australia d...","Person 1: ""Are you set for your upcoming comed...","Person 1: ""Hey, how's your day shaping up?""\nP..."
7,8,"Person 1: ""Hey! What's up?""\nPerson 2: ""Oh, ju...","Person 1: ""Hey, I started listening to Avatar,...","Person 1: ""I actually picked up 'IT' again for...","Person 1: ""Know of any good rock bands droppin...","Person 1: ""Hey! What's up?""\nPerson 2: ""Oh, ju..."
8,9,"Person 1: ""Hey there, thinking of hitting up a...","Person 1: ""Just letting you know, I made it sa...","Person 1: ""I took your advice and left my frie...","Person 1: ""Caught some Japanese wrestling show...","Person 1: ""Hey there, thinking of hitting up a..."
9,10,"Person 1: ""I finally tackled that mess in my h...","Person 1: ""How about we seize a day for a hike...","Person 1: ""Have you explored any parks or vent...","Person 1: ""Absolutely, I love taking a dip whe...","Person 1: ""I finally tackled that mess in my h..."


### Key Fact Generation

In [ ]:
def key_fact_generation(data: pd.DataFrame, data_path: str, file_name: str) -> pd.DataFrame:
    """
    Generate facts from the conversation using GPT-4 by taking data from the provided DataFrame,
    ensuring not to reprocess data already in key_facts.xlsx.

    Parameters:
    - data (DataFrame): The data containing conversations to be analyzed for facts.
    - data_path (str): The path to the directory where the file is or will be stored.
    - file_name (str): The name of the file where the data is or will be stored.

    Returns:
    - DataFrame: A dataframe with generated facts.
    """

    # Load existing key facts data if it exists and extract existing IDs.
    try:
        existing_key_facts = pd.read_excel(data_path + file_name)
        existing_ids = set(existing_key_facts['id'].values)
    except FileNotFoundError:
        existing_ids = set()

    key_facts = pd.DataFrame(columns=['id', 's1_facts_p1', 's1_facts_p2', 's2_facts_p1', 's2_facts_p2', 's3_facts_p1', 's3_facts_p2', 's4_facts_p1', 's4_facts_p2'])

    # Extract facts for each session in the provided data
    for idx, row in data.iterrows():
        if row['id'] not in existing_ids:
            fact_row = {'id': row['id']}
            for session_num, col in enumerate(['s1_conv', 's2_conv', 's3_conv', 's4_conv'], 1):
                for person_num in [1, 2]:  # Iterate for both Person 1 and Person 2
                    fact_extraction_prompt = (
                    f"Given the conversation below, list out all the facts that we can collect about Person {person_num}. "
                    f"I want facts that have been explicitly said, not inferred. These facts can range from what the "
                    f"person thinks about a certain topic, the person's name, family relationships, interests, "
                    f"educational/career interests, travel experiences, their culture and so on.\n\n"
                    f"Do not provide any other explanation but just give the list in bullet points. "
                    f"Explicity state that the fact is about Person {person_num} when listing it out."
                    f"Do not write anything before the output of facts such as “Sure! Here are the facts about "
                    f"Person {person_num} that we can gather from the conversation:” or \"Facts about Person {person_num}:\" Just get to the point.\n\n"
                    f"The output should be a simple list of facts separated with a - and at the end of each fact, "
                    f"it should be separated with a new line.\nIf there is no information on a given topic I mentioned, "
                    f"you do not need to output anything about it. Also, you do not need to ask me if there is any clarification. "
                    f"Only output the list as well as the line number you found the information from in parentheses.\n"
                    f"An example of an element of a list would look like this. - Person {person_num} thinks that retro foods like peanut "
                    f"butter and banana toast or celery with peanut butter and raisins could be popular. (2)\n\n Dialogue:\n"
                    f"{row[col]}"
                )
                    response = openai.ChatCompletion.create(
                        model="gpt-4",
                        messages=[{"role": "system", "content": fact_extraction_prompt}]
                    )
                    fact_row[f's{session_num}_facts_p{person_num}'] = response['choices'][0]['message']['content']

            key_facts = pd.concat([key_facts, pd.DataFrame([fact_row])], ignore_index=True)

    return key_facts


def append_and_save_keyfacts(data_path: str, file_name: str, new_key_facts_data: pd.DataFrame) -> pd.DataFrame:
    """
    Appends new key facts to the existing key facts data and saves to file.

    Parameters:
    - data_path (str): Path to the directory where the file is or will be stored.
    - file_name (str): The name of the file where the data is or will be stored.
    - new_key_facts_data (DataFrame): The new key facts data.

    Returns:
    - DataFrame: The appended dataframe.
    """

    # Load existing key facts data if it exists
    try:
        existing_key_facts_data = pd.read_excel(data_path + file_name)
        combined_data = pd.concat([existing_key_facts_data, new_key_facts_data], ignore_index=True)
    except FileNotFoundError:
        combined_data = new_key_facts_data

    # Ensure proper order
    column_order = ['id', 's1_facts_p1', 's1_facts_p2', 's2_facts_p1', 's2_facts_p2', 's3_facts_p1', 's3_facts_p2', 's4_facts_p1', 's4_facts_p2']
    combined_data = combined_data[column_order]

    # Sort and save to file
    combined_data = combined_data.sort_values(by='id').reset_index(drop=True)
    combined_data.to_excel(data_path + file_name, index=False)

    return combined_data

In [ ]:
keyfacts_file_name = "key_facts_gpt4.xlsx"
augmented_file_name = "augmented_conv_gpt4.xlsx"
augmented_data = pd.read_excel(data_path + "Augmented GPT4/" + augmented_file_name)

while True:
    key_facts = key_fact_generation(augmented_data, data_path + "Augmented GPT4/", keyfacts_file_name)
    if not key_facts.empty:  # If there's new data to be saved
        result_data = append_and_save_keyfacts(data_path + "Augmented GPT4/", keyfacts_file_name, key_facts)
    key_facts = pd.read_excel(data_path + "Augmented GPT4/" + keyfacts_file_name)

    # Check for NaN values in the dataframe
    if key_facts.isna().any().any():
        # Drop rows containing NaN values
        key_facts.dropna(inplace=True)
        key_facts.to_excel(data_path + "Augmented GPT4/" + keyfacts_file_name, index=False)
    else:
        # If there are no NaN values, break the loop
        break

display(key_facts)

,id,s1_facts_p1,s1_facts_p2,s2_facts_p1,s2_facts_p2,s3_facts_p1,s3_facts_p2,s4_facts_p1,s4_facts_p2
0,1,- Person 1's name is Brian. (1)\n- Brian fanta...,- Person 2 feels good. (2)\n- Person 2 plans t...,- Person 1 was listening to Bob Dylan music in...,- Person 2 is a fan of Bob Dylan's music. (2)\...,- Person 1 applied for an internship in an art...,- Person 2 showed interest in hearing back abo...,- Person 1 received an email from their doctor...,- Person 2 has allergies and appears to experi...
1,2,- Person 1 is looking forward to the Fall seas...,- Person 2's favorite season is fall. (2)\n- P...,- Person 1 asked Person 2 about their doctoral...,- Person 2 is in the initial stages of an educ...,- Person 1 is supporting Person 2 in their PhD...,- Person 2 is currently pursuing a PhD study. ...,- Person 1 was at a coffee shop where they had...,- Person 2 is planning a bike ride next Saturd...
2,3,- Person 1 enjoys having a good drink. (1)\n- ...,- Person 2 needs a drink because her husband j...,- Person 1 has a chihuahua. (1)\n- Person 1 en...,- Person 2 finds the idea of a dog wearing a t...,- Person 1 has offered to spend time with Pers...,- Person 2 has children. (2)\n- Person 2's chi...,- Person 1 borrowed both Frozen movies for Per...,- Person 2 has multiple daughters who love the...
3,4,- Person 1 experienced sunny weather. (1)\n- P...,- Person 2 enjoys the autumn season for its co...,- Person 1 recently cooked an Italian feast fo...,- Person 2's favorite Italian dish is pizza. (...,- Person 1 is installing a new pizza oven at t...,- Person 2 shows interest in the new oven bein...,- Person 1 prefers thin crust on pizzas. (3)\n...,- Person 2 prefers thin crust on their pizzas....
4,5,- Person 1 could have gone pro in tennis but i...,- Person 2 is doing good. (2)\n- Person 2 aske...,- Person 1 recently got their laptop fixed. (1...,- Person 2 shows interest about Person 1's lap...,- Person 1 asked if Person 2 managed to diagno...,- Person 2 has been unable to diagnose issues ...,- Person 1 helped to resolve an issue with Per...,- Person 2 says their laptop is running much b...
5,6,- Person 1 asked about Person 2's day. (1)\n- ...,- Person 2's day is going well. (2)\n- Person ...,- Person 1 is watching a concert featuring a p...,- Person 2 used to play in a pop punk band. (2...,- Person 1 suggested hanging out with local mu...,- Person 2 has tried to form a music club with...,- Person 1 has recently checked out Person 2's...,- Person 2 is part of a music group that has a...
6,7,- Person 1 asked about Person 2's day. (1)\n- ...,- Person 2's day is going well. (2)\n- Person ...,- Person 1 tried diving for the first time and...,- Person 2 went diving the previous weekend. (...,"- Person 1 is interested in diving, particular...",- Person 2 has been on a diving expedition in ...,- Person 1 has family who are interested in co...,- Person 2 has an upcoming comedy gig at a loc...
7,8,- Person 1 is interested in NFL fantasy footba...,- Person 2 has recently finished a gaming sess...,- Person 1 listens to the band Avatar. (1)\n- ...,- Person 2 likes the band Avatar. (2)\n- Perso...,- Person 1 is re-reading the book 'IT'. (1)\n-...,- Person 2 has not watched the recent global r...,- Person 1 is interested in rock bands. (1)\n-...,- Person 2 hasn't been able to keep up with co...
8,9,- Person 1 asked if Person 2 is going to any p...,- Person 2 does not want to attend any parties...,- Person 1 went to a party. (1)\n- Person 1 dr...,- Person 2 is concerned about Person 1's safet...,- Person 1 recently left a party early on advi...,- Person 2 gave Person 1 advice on sleep being...,- Person 1 watched some Japanese wrestling sho...,- Person 2 asked where Person 1 was watching t...
9,10,- Person 1 cleaned their house for four hours....,- Person 2 congratulates Person 1 on completin...,- Person 1 wants to go for a hike and some cof...,- Person 2 was asked to go hiking and have cof...,- Person 1 is interested in exploring parks an...,- Perso

### False Facts

In [ ]:
def false_fact_generation(data: pd.DataFrame, data_path: str, file_name: str) -> pd.DataFrame:
    """
    Generate false facts from the conversation using GPT-4 by taking data from the provided DataFrame,
    ensuring not to reprocess data already in false_facts.xlsx.

    Parameters:
    - data (DataFrame): The data containing conversations to be analyzed for false facts.
    - data_path (str): The path to the directory where the file is or will be stored.
    - file_name (str): The name of the file where the data is or will be stored.

    Returns:
    - DataFrame: A dataframe with generated false facts.
    """

    # Load existing false facts data if it exists and extract existing IDs.
    try:
        existing_key_facts = pd.read_excel(data_path + file_name)
        existing_ids = set(existing_key_facts['id'].values)
    except FileNotFoundError:
        existing_ids = set()

    false_facts = pd.DataFrame(columns=['id', 's1_false_facts_p1', 's1_false_facts_p2', 's2_false_facts_p1', 's2_false_facts_p2', 's3_false_facts_p1', 's3_false_facts_p2', 's4_false_facts_p1', 's4_false_facts_p2'])

    # Extract facts for each session in the provided data
    for idx, row in data.iterrows():
        if row['id'] not in existing_ids:
            fact_row = {'id': row['id']}
            for session_num, col in enumerate(['s1_conv', 's2_conv', 's3_conv', 's4_conv'], 1):
                for person_num in [1, 2]:  # Iterate for both Person 1 and Person 2
                    fact_extraction_prompt = (
                    f"Below is a conversation between Person 1 and Person 2 I want you to come up with a list of false "
                    f"facts about Person {person_num} that are on topic with what the conversation is about but wrong "
                    f"factually. Come up with at least 10 false facts about Person {person_num}, for each false fact, have line "
                    f"numbers in () showing where the topic is mentioned.  I want facts that have been explicitly said, "
                    f"not inferred and then falsified. \n\n Do not provide any other explanation but just give the list in bullet points. "
                    f"Explicity state that the false fact is about Person {person_num} when listing it out, but do not state that it is false."
                    f"Do not write anything before the output of facts such as “Sure! Here are the false facts about "
                    f"Person {person_num} that we can gather from the conversation:” or \"False facts about Person "
                    f"{person_num}:\ Just get to the point.\n\n The output should be a simple list of facts separated "
                    f"with a - and at the end of each fact, it should be separated with a new line.\n Also, you do not "
                    f"need to ask me if there is any clarification.  Only output the list as well as the line number "
                    f"you found the information from in parentheses.\n An example of an element of a list would look "
                    f"like this. - Person {person_num} thinks that retro foods like peanut butter and banana toast or "
                    f"celery with peanut butter and raisins could be popular. (2)"
                    f"\n\n Dialogue:\n {row[col]}"
                )
                    response = openai.ChatCompletion.create(
                        model="gpt-4",
                        messages=[{"role": "system", "content": fact_extraction_prompt}]
                    )
                    fact_row[f's{session_num}_false_facts_p{person_num}'] = response['choices'][0]['message']['content']

            false_facts = pd.concat([false_facts, pd.DataFrame([fact_row])], ignore_index=True)

    return false_facts


def append_and_save_falsefacts(data_path: str, file_name: str, new_false_facts_data: pd.DataFrame) -> pd.DataFrame:
    """
    Appends new false facts to the existing false facts data and saves to file.

    Parameters:
    - data_path (str): Path to the directory where the file is or will be stored.
    - file_name (str): The name of the file where the data is or will be stored.
    - new_false_facts_data (DataFrame): The new false facts data.

    Returns:
    - DataFrame: The appended dataframe.
    """

    # Load existing false facts data if it exists
    try:
        existing_false_facts_data = pd.read_excel(data_path + file_name)
        combined_data = pd.concat([existing_false_facts_data, new_false_facts_data], ignore_index=True)
    except FileNotFoundError:
        combined_data = new_false_facts_data

    # Ensure proper order
    column_order = ['id', 's1_false_facts_p1', 's1_false_facts_p2', 's2_false_facts_p1', 's2_false_facts_p2', 's3_false_facts_p1', 's3_false_facts_p2', 's4_false_facts_p1', 's4_false_facts_p2']
    combined_data = combined_data[column_order]

    # Sort and save to file
    combined_data = combined_data.sort_values(by='id').reset_index(drop=True)
    combined_data.to_excel(data_path + file_name, index=False)

    return combined_data

In [ ]:
falsefacts_file_name = "false_facts_gpt4.xlsx"
augmented_file_name = "augmented_conv_gpt4.xlsx"
augmented_data = pd.read_excel(data_path + "Augmented GPT4/" + augmented_file_name)

while True:
    false_facts = false_fact_generation(augmented_data, data_path + "Augmented GPT4/", falsefacts_file_name)
    if not false_facts.empty:  # If there's new data to be saved
        result_data = append_and_save_falsefacts(data_path + "Augmented GPT4/", falsefacts_file_name, false_facts)
    false_facts = pd.read_excel(data_path + "Augmented GPT4/" + falsefacts_file_name)

    # Check for NaN values in the dataframe
    if false_facts.isna().any().any():
        # Drop rows containing NaN values
        false_facts.dropna(inplace=True)
        false_facts.to_excel(data_path + "Augmented GPT4/" + falsefacts_file_name, index=False)
    else:
        # If there are no NaN values, break the loop
        break

display(false_facts)

,id,s1_false_facts_p1,s1_false_facts_p2,s2_false_facts_p1,s2_false_facts_p2,s3_false_facts_p1,s3_false_facts_p2,s4_false_facts_p1,s4_false_facts_p2
0,1,- Person 1 is named John. (1)\n- Person 1 pref...,- Person 2 doesn't like hiking. (2)\n- Person ...,- Person 1 has attended a Bob Dylan concert. (...,- Person 2 dislikes Bob Dylan's music. (2)\n- ...,- Person 1 has never applied for an internship...,- Person 2 does not work for a marketing firm....,- Person 1 has allergies. (1)\n- Person 1 is a...,- Person 2's doctor has already informed them ...
1,2,- Person 1 dislikes Fall. (1)\n- Person 1 is c...,- Person 2 dislikes the Fall season. (2)\n- Pe...,- Person 1 is an established author of thrille...,- Person 2 has completed their doctoral resear...,- Person 1 is pursuing a PhD. (1)\n- Person 1 ...,- Person 2 already has a PhD. (1)\n- Person 2 ...,- Person 1 tried a caramel cappuccino scone fo...,- Person 2 dislikes caramel cappuccino scones....
2,3,- Person 1 dislikes drinking. (1)\n- Person 1'...,"- Person 2 does not have any children. (4, 6, ...",- Person 1's chihuahua doesn't like being dres...,- Person 2 has a Chihuahua. (2)\n- Person 2's ...,- Person 1 doesn't have a dog. (3)\n- Person 1...,- Person 2 is unemployed. (5) \n- Person 2 aba...,- Person 1 doesn't have the Frozen movies yet....,- Person 2 is not a fan of Frozen movies. (2)\...
3,4,- Person 1 dislikes the sunny weather. (1)\n- ...,- Person 2 dislikes autumn and its colors. (2)...,"- Person 1 does not like Italian cuisine. (1,3...",- Person 2 dislikes pizza. (2)\n- Person 2 wor...,- Person 1's restaurant has been using a brick...,- Person 2 owns the restaurant where the new p...,- Person 1 prefers thick crust on their pizzas...,- Person 2 dislikes thin crust on pizzas. (2)\...
4,5,- Person 1 is a professional tennis player. (3...,- Person 2 has a deep dislike for tennis. (2)\...,- Person 1's laptop was not broken. (1)\n- Per...,- Person 2 fixed Person 1's laptop. (1)\n- Per...,- Person 1 is a professional computer technici...,- Person 2 is an expert in diagnosing laptop i...,- Person 1 never helped Person 2 with their la...,- Person 2 thought that defragmenting a disk w...
5,6,- Person 1 started their day with a pizza for ...,- Person 2 is a woman. (18)\n- Person 2 has ne...,- Person 1 is a member of a pop punk band. (2)...,- Person 2 is currently in a pop punk band. (2...,- Person 1 is a member of a local music band. ...,- Person 2 has never tried forming an informal...,- Person 1 is a professional musician on the s...,- Person 2 doesn't play any music. (2)\n- Pers...
6,7,- Person 1 has been coaching soccer for five y...,- Person 2 is a professional event planner. (4...,- Person 1 has been diving for several years. ...,- Person 2 thinks that diving can go up to 300...,- Person 1 has gone on a diving expedition in ...,- Person 2 is a professional diver. (2)\n- Per...,- Person 1 is the one doing a comedy gig. (1)\...,- Person 2 has never performed a comedy gig be...
7,8,- Person 1 is a musician. (2)\n- Person 1 enjo...,- Person 2 dislikes sports. (4)\n- Person 2’s ...,- Person 1 started listening to Avatar on Pers...,- Person 2 has never been to a Bulldogs game. ...,- Person 1 has only read the book 'IT' once. (...,- Person 2 has seen the recent global release ...,- Person 1 is not interested in rock bands. (1...,- Person 2 regularly follows concert news. (1-...
8,9,- Person 1 doesn't enjoy attending parties. (1...,- Person 2 is going to a party tonight despite...,- Person 1 does not enjoy playing Cards Agains...,- Person 2 attended the same party as Person 1...,- Person 1 left the friend's party late that n...,- Person 2 left the party early. (1)\n- Person...,- Person 1 dislikes Japanese wrestling. (1)\n-...,- Person 2 was watching wrestling at a local b...
9,10,- Person 1 spent three hours cleaning their ho...,- Person 2 did not clean their house. (2)\n- P...,- Person 1 dislikes coffee. (1)\n- Person 1 ha...,- Person 2 dislikes hiking. (4)\n- Person 2 di...,- Perso

### Facts Dataset

In [ ]:
keyfacts_file_name = "key_facts_gpt4.xlsx"
key_facts = pd.read_excel(data_path + "Augmented GPT4/" + keyfacts_file_name)
falsefacts_file_name = "false_facts_gpt4.xlsx"
false_facts = pd.read_excel(data_path + "Augmented GPT4/" + falsefacts_file_name)
facts = key_facts.merge(false_facts, on='id')
for i in range(1, 5):
    facts[f's{i}_facts'] = facts[f's{i}_facts_p1'] + "\n" + facts[f's{i}_facts_p2']
    facts[f's{i}_false_facts'] = facts[f's{i}_false_facts_p1'] + "\n" + facts[f's{i}_false_facts_p2']
columns_to_drop = [f's{i}_{fact_type}_p{j}' for i in range(1, 5) for fact_type in ['facts', 'false_facts'] for j in range(1, 3)]
facts = facts.drop(columns=columns_to_drop)
facts_file_name = "facts_gpt4.xlsx"
facts.to_excel(data_path + "Augmented GPT4/" + facts_file_name, index=False)
facts = pd.read_excel(data_path + "Augmented GPT4/" + facts_file_name)

display(facts)

,id,s1_facts,s1_false_facts,s2_facts,s2_false_facts,s3_facts,s3_false_facts,s4_facts,s4_false_facts
0,1,- Person 1's name is Brian. (1)\n- Brian fanta...,- Person 1 is named John. (1)\n- Person 1 pref...,- Person 1 was listening to Bob Dylan music in...,- Person 1 has attended a Bob Dylan concert. (...,- Person 1 applied for an internship in an art...,- Person 1 has never applied for an internship...,- Person 1 received an email from their doctor...,- Person 1 has allergies. (1)\n- Person 1 is a...
1,2,- Person 1 is looking forward to the Fall seas...,- Person 1 dislikes Fall. (1)\n- Person 1 is c...,- Person 1 asked Person 2 about their doctoral...,- Person 1 is an established author of thrille...,- Person 1 is supporting Person 2 in their PhD...,- Person 1 is pursuing a PhD. (1)\n- Person 1 ...,- Person 1 was at a coffee shop where they had...,- Person 1 tried a caramel cappuccino scone fo...
2,3,- Person 1 enjoys having a good drink. (1)\n- ...,- Person 1 dislikes drinking. (1)\n- Person 1'...,- Person 1 has a chihuahua. (1)\n- Person 1 en...,- Person 1's chihuahua doesn't like being dres...,- Person 1 has offered to spend time with Pers...,- Person 1 doesn't have a dog. (3)\n- Person 1...,- Person 1 borrowed both Frozen movies for Per...,- Person 1 doesn't have the Frozen movies yet....
3,4,- Person 1 experienced sunny weather. (1)\n- P...,- Person 1 dislikes the sunny weather. (1)\n- ...,- Person 1 recently cooked an Italian feast fo...,"- Person 1 does not like Italian cuisine. (1,3...",- Person 1 is installing a new pizza oven at t...,- Person 1's restaurant has been using a brick...,- Person 1 prefers thin crust on pizzas. (3)\n...,- Person 1 prefers thick crust on their pizzas...
4,5,- Person 1 could have gone pro in tennis but i...,- Person 1 is a professional tennis player. (3...,- Person 1 recently got their laptop fixed. (1...,- Person 1's laptop was not broken. (1)\n- Per...,- Person 1 asked if Person 2 managed to diagno...,- Person 1 is a professional computer technici...,- Person 1 helped to resolve an issue with Per...,- Person 1 never helped Person 2 with their la...
5,6,- Person 1 asked about Person 2's day. (1)\n- ...,- Person 1 started their day with a pizza for ...,- Person 1 is watching a concert featuring a p...,- Person 1 is a member of a pop punk band. (2)...,- Person 1 suggested hanging out with local mu...,- Person 1 is a member of a local music band. ...,- Person 1 has recently checked out Person 2's...,- Person 1 is a professional musician on the s...
6,7,- Person 1 asked about Person 2's day. (1)\n- ...,- Person 1 has been coaching soccer for five y...,- Person 1 tried diving for the first time and...,- Person 1 has been diving for several years. ...,"- Person 1 is interested in diving, particular...",- Person 1 has gone on a diving expedition in ...,- Person 1 has family who are interested in co...,- Person 1 is the one doing a comedy gig. (1)\...
7,8,- Person 1 is interested in NFL fantasy footba...,- Person 1 is a musician. (2)\n- Person 1 enjo...,- Person 1 listens to the band Avatar. (1)\n- ...,- Person 1 started listening to Avatar on Pers...,- Person 1 is re-reading the book 'IT'. (1)\n-...,- Person 1 has only read the book 'IT' once. (...,- Person 1 is interested in rock bands. (1)\n-...,- Person 1 is not interested in rock bands. (1...
8,9,- Person 1 asked if Person 2 is going to any p...,- Person 1 doesn't enjoy attending parties. (1...,- Person 1 went to a party. (1)\n- Person 1 dr...,- Person 1 does not enjoy playing Cards Agains...,- Person 1 recently left a party early on advi...,- Person 1 left the friend's party late that n...,- Person 1 watched some Japanese wrestling sho...,- Person 1 dislikes Japanese wrestling. (1)\n-...
9,10,- Person 1 cleaned their house for four hours....,- Person 1 spent three hours cleaning their ho...,- Person 1 wants to go for a hike and some cof...,- Person 1 dislikes coffee. (1)\n- Person 1 ha...,- Person 1 is interested in exploring parks an...,- Per

### Full Dataset

In [ ]:
augmented_file_name = "augmented_conv_gpt4.xlsx"
augmented_data = pd.read_excel(data_path + "Augmented GPT4/" + augmented_file_name)
facts_file_name = "facts_gpt4.xlsx"
facts = pd.read_excel(data_path + "Augmented GPT4/" + facts_file_name)
fulldataset_file_name = "full_dataset_gpt4.xlsx"
full_dataset = augmented_data.merge(facts, on='id')
full_dataset.to_excel(data_path + "Augmented GPT4/" + fulldataset_file_name, index=False)
full_dataset = pd.read_excel(data_path + "Augmented GPT4/" + fulldataset_file_name)

display(full_dataset)

,id,s1_conv,s2_conv,s3_conv,s4_conv,full_conv,s1_facts,s1_false_facts,s2_facts,s2_false_facts,s3_facts,s3_false_facts,s4_facts,s4_false_facts
0,1,"Person 1: ""Hey there, I'm Brian. How's your da...","Person 1: ""Funny you mention it, I was jamming...","Person 1: ""Guess what? I took your advice and ...","Person 1: ""Speaking of allergies, I just got a...","Person 1: ""Hey there, I'm Brian. How's your da...",- Person 1's name is Brian. (1)\n- Brian fanta...,- Person 1 is named John. (1)\n- Person 1 pref...,- Person 1 was listening to Bob Dylan music in...,- Person 1 has attended a Bob Dylan concert. (...,- Person 1 applied for an internship in an art...,- Person 1 has never applied for an internship...,- Person 1 received an email from their doctor...,- Person 1 has allergies. (1)\n- Person 1 is a...
1,2,"Person 1: ""I can't wait for Fall! Finally, an ...","Person 1: ""So, what's your doctoral research t...","Person 1: ""How's that PhD study coming along?""...","Person 1: ""Guess what they had at the coffee s...","Person 1: ""I can't wait for Fall! Finally, an ...",- Person 1 is looking forward to the Fall seas...,- Person 1 dislikes Fall. (1)\n- Person 1 is c...,- Person 1 asked Person 2 about their doctoral...,- Person 1 is an established author of thrille...,- Person 1 is supporting Person 2 in their PhD...,- Person 1 is pursuing a PhD. (1)\n- Person 1 ...,- Person 1 was at a coffee shop where they had...,- Person 1 tried a caramel cappuccino scone fo...
2,3,"Person 1: ""I'm quite partial to a good drink m...","Person 1: ""The other day I got my chihuahua al...","Person 1: ""So, how are things with you and the...","Person 1: ""Remembered that your girls love Fro...","Person 1: ""I'm quite partial to a good drink m...",- Person 1 enjoys having a good drink. (1)\n- ...,- Person 1 dislikes drinking. (1)\n- Person 1'...,- Person 1 has a chihuahua. (1)\n- Person 1 en...,- Person 1's chihuahua doesn't like being dres...,- Person 1 has offered to spend time with Pers...,- Person 1 doesn't have a dog. (3)\n- Person 1...,- Person 1 borrowed both Frozen movies for Per...,- Person 1 doesn't have the Frozen movies yet....
3,4,"Person 1: ""Hello! How's the climate treating y...","Person 1: ""Just wrapped cooking up an Italian ...","Person 1: ""We're installing a new pizza oven a...","Person 1: ""What kind of pizza crust do you pre...","Person 1: ""Hello! How's the climate treating y...",- Person 1 experienced sunny weather. (1)\n- P...,- Person 1 dislikes the sunny weather. (1)\n- ...,- Person 1 recently cooked an Italian feast fo...,"- Person 1 does not like Italian cuisine. (1,3...",- Person 1 is installing a new pizza oven at t...,- Person 1's restaurant has been using a brick...,- Person 1 prefers thin crust on pizzas. (3)\n...,- Person 1 prefers thick crust on their pizzas...
4,5,"Person 1: ""Hey, how are you holding up tonight...","Person 1: ""Good news! Finally got my laptop fi...","Person 1: ""Did you manage to diagnose your lap...","Person 1: ""How's the laptop now? Did my interv...","Person 1: ""Hey, how are you holding up tonight...",- Person 1 could have gone pro in tennis but i...,- Person 1 is a professional tennis player. (3...,- Person 1 recently got their laptop fixed. (1...,- Person 1's laptop was not broken. (1)\n- Per...,- Person 1 asked if Person 2 managed to diagno...,- Person 1 is a professional computer technici...,- Person 1 helped to resolve an issue with Per...,- Person 1 never helped Person 2 with their la...
5,6,"Person 1: ""Hey, how's your day going?""\nPerson...","Person 1: ""Watching this concert right now and...","Person 1: ""Ever consider hanging out with othe...","Person 1: ""Checked out your YouTube channel re...","Person 1: ""Hey, how's your day going?""\nPerson...",- Person 1 asked about Person 2's day. (1)\n- ...,- Person 1 started their day with a pizza for ...,- Person 1 is watching a concert featuring a p...,- Person 1 is a member of a pop punk band. (2)...,- Person 1 suggested hanging out with l